In [1]:
!pip install -q plotnine
from plotnine import *


In [79]:
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [2]:
import numpy as np
import pandas as pd 

#from subprocess import check_output #it was in code that I got, it might be usefull for windows users
#print(check_output(["ls", "."]).decode("utf8"))

# read the data

bg_dt = pd.read_csv('all_2400.H.tsv.gz', compression='gzip', 
                                 header=0, sep='\t', index_col = 0) #read data
bg_dt2 = bg_dt.T # transpose data
bg_dt2.index.name = 'SampleID' #change index name to merge later with the manifest
bg_dt2.iloc[1:10,1:10] # check into first ten rows, all looks ok

chr_pos_ref_alt,1:58771:T:C,1:183401:C:G,1:186291:G:A,1:281912:C:G,1:531693:C:T,1:663390:T:C,1:689966:T:C,1:734153:G:C,1:813885:G:A
SampleID,,,,,,,,,
HG00097,1|1,0|0,0|0,0|0,1|1,0|0,0|0,0|0,1|1
HG00099,0|0,0|0,1|0,0|0,0|1,0|0,0|0,0|0,1|1
HG00100,1|0,0|0,0|0,0|0,1|0,0|0,0|0,0|0,1|1
HG00101,1|0,0|0,0|0,1|0,0|1,0|0,0|0,0|0,1|0
HG00102,1|1,0|0,0|0,1|0,0|0,0|0,0|0,1|0,1|1
HG00103,0|0,0|0,0|0,0|0,1|1,0|0,0|0,0|0,1|1
HG00105,0|1,0|0,0|0,1|0,0|0,0|0,0|0,0|0,1|1
HG00106,0|1,0|1,0|0,0|0,0|1,0|0,1|0,0|1,1|0
HG00107,1|0,0|0,0|0,1|0,1|1,1|0,0|0,0|0,0|0


In [3]:

# read the manifest file

manifest = pd.read_csv('manifest.txt', sep = " ", index_col= 1) # read the manifest
print(manifest.head) # fyi how it looks
manifest = manifest.loc[:,'Population'] # leave only Pop
print(manifest.head) # looks fine
bg_dt3 = pd.merge(manifest,bg_dt2, left_index=True, right_index=True) # merge with the data by indices
bg_dt3

<bound method NDFrame.head of          FamilyID FatherID MotherID  Sex Population Superpopulation
SampleID                                                           
HG00096   HG00096        0        0    1        GBR             EUR
HG00097   HG00097        0        0    2        GBR             EUR
HG00099   HG00099        0        0    2        GBR             EUR
HG00100   HG00100        0        0    2        GBR             EUR
HG00101   HG00101        0        0    1        GBR             EUR
...           ...      ...      ...  ...        ...             ...
NA21137   NA21137        0        0    2        GIH             SAS
NA21141   NA21141        0        0    2        GIH             SAS
NA21142   NA21142        0        0    2        GIH             SAS
NA21143   NA21143        0        0    2        GIH             SAS
NA21144   NA21144        0        0    2        GIH             SAS

[3202 rows x 6 columns]>
<bound method NDFrame.head of SampleID
HG00096    GBR
HG0009

,Population,1:17222:A:G,1:58771:T:C,1:183401:C:G,1:186291:G:A,1:281912:C:G,1:531693:C:T,1:663390:T:C,1:689966:T:C,1:734153:G:C,...,22:50509881:A:G,22:50555369:G:C,22:50592780:G:C,22:50665742:A:G,22:50717650:G:A,22:50740572:C:A,22:50768105:T:C,22:50768106:T:G,22:50787916:C:T,22:50807426:C:T
SampleID,,,,,,,,,,,,,,,,,,,,,
HG00096,GBR,0|0,1|0,0|0,0|0,0|0,1|1,0|0,0|0,0|0,...,0|0,1|1,1|1,1|1,0|0,0|1,1|0,0|0,1|0,0|0
HG00097,GBR,0|0,1|1,0|0,0|0,0|0,1|1,0|0,0|0,0|0,...,0|0,1|1,1|1,0|0,0|0,1|0,0|1,0|0,0|1,0|0
HG00099,GBR,0|0,0|0,0|0,1|0,0|0,0|1,0|0,0|0,0|0,...,1|1,0|0,0|1,1|0,1|0,0|0,1|1,0|0,1|0,0|0
HG00100,GBR,0|0,1|0,0|0,0|0,0|0,1|0,0|0,0|0,0|0,...,1|1,0|0,1|0,1|1,1|0,0|0,0|1,0|0,0|0,0|0
HG00101,GBR,0|1,1|0,0|0,0|0,1|0,0|1,0|0,0|0,0|0,...,0|1,0|0,0|1,0|1,1|1,0|0,0|1,0|0,0|1,0|0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA21137,GIH,0|1,0|1,0|0,1|0,0|1,1|1,0|0,0|0,0|0,...,0|0,0|0,1|0,1|1,0|0,1|0,1|1,0|0,0|1,0|0
NA21141,GIH,0|0,1|0,0|1,0|0,0|0,0|1,0|0,0|0,0|0,...,0|0,0|1,1|1,1|0,0|0,0|1,0|0,0|0,1|0,0|0
NA21142,GIH,0|1,1|1,0|0,0|0,0|0,0|1,0|0,0|0,0|0,...,0|0,0|1,0|1,0|1,0|0,1|1,0|0,0|0,0|1,0|1


In [17]:

# read the samples that fall into train

smpl_list = pd.read_csv('samples.2561.txt', header=None, index_col = 0) # read samples to be subsetted
print(smpl_list.head) # check how they look, they are indexes here
print(bg_dt3.shape) # check the shape of the previos data, 3202 samples and 62.9K SNPs + Pop

<bound method NDFrame.head of Empty DataFrame
Columns: []
Index: [HG02330, HG02471, HG02470, HG02580, HG01896, HG01887, HG02511, HG01985, HG01882, HG02308, HG01888, HG02479, HG02258, HG02145, HG02545, HG02501, HG02477, HG02323, HG01960, HG02497, HG02554, HG02496, HG01879, HG02143, HG02557, HG01989, HG02144, HG01956, HG02095, HG02325, HG02107, HG02489, HG02485, HG01886, HG01986, HG01883, HG02419, HG02052, HG02282, HG02508, HG02455, HG02439, HG02536, HG02010, HG02255, HG01915, HG02011, HG02318, HG02334, HG01987, HG01916, HG01914, HG02280, HG02314, HG02009, HG02322, HG01959, HG01890, HG01912, HG02257, HG02420, HG01885, HG02332, HG02051, HG02476, HG01958, HG02537, HG02316, HG02450, HG02427, HG02111, HG02281, HG02429, HG01889, HG02546, HG01990, HG02055, HG02315, HG02012, HG02442, HG02108, HG01891, HG02337, HG01897, HG02555, HG02558, HG02481, HG02256, HG02502, HG01880, HG02317, HG01988, HG02343, NA19828, NA19908, NA20321, NA20274, NA20356, NA20332, NA19984, ...]

[2561 rows x 0 columns]>
(32

In [55]:

# leave only train, safe Y train

train=bg_dt3[bg_dt3.index.isin(smpl_list.index)] # subset the train via train samples
train_Y = train.loc[:,'Population'] # get Y stay alone
train_Y = train_Y.to_frame()
train_Y.index = range(0,2561) # change to merge it later

train = train.drop('Population', axis = 1) # remove Y from data
print(train.iloc[0:10,0:10]) # check that all is fine
print(train_Y)

         1:17222:A:G 1:58771:T:C 1:183401:C:G 1:186291:G:A 1:281912:C:G  \
SampleID                                                                  
HG00096          0|0         1|0          0|0          0|0          0|0   
HG00099          0|0         0|0          0|0          1|0          0|0   
HG00100          0|0         1|0          0|0          0|0          0|0   
HG00101          0|1         1|0          0|0          0|0          1|0   
HG00102          0|0         1|1          0|0          0|0          1|0   
HG00103          0|0         0|0          0|0          0|0          0|0   
HG00105          0|0         0|1          0|0          0|0          1|0   
HG00107          0|0         1|0          0|0          0|0          1|0   
HG00108          0|0         0|1          0|0          0|0          0|0   
HG00109          0|0         0|0          0|0          0|0          0|0   

         1:531693:C:T 1:663390:T:C 1:689966:T:C 1:734153:G:C 1:813885:G:A  
SampleID               

In [20]:

# recode all genotypes to 0 1 2

train = train.replace(['0|0'], '0')
train = train.replace(['0|1',"1|0"], '1')
train = train.replace(['1|1'], '2')
print(train.iloc[0:10,0:10])

         1:17222:A:G 1:58771:T:C 1:183401:C:G 1:186291:G:A 1:281912:C:G  \
SampleID                                                                  
HG00096            0           1            0            0            0   
HG00099            0           0            0            1            0   
HG00100            0           1            0            0            0   
HG00101            1           1            0            0            1   
HG00102            0           2            0            0            1   
HG00103            0           0            0            0            0   
HG00105            0           1            0            0            1   
HG00107            0           1            0            0            1   
HG00108            0           1            0            0            0   
HG00109            0           0            0            0            0   

         1:531693:C:T 1:663390:T:C 1:689966:T:C 1:734153:G:C 1:813885:G:A  
SampleID               

In [21]:

# simple normalisation, I guess I could use StandartScaller, but anyway

train = train.apply(pd.to_numeric) # to numeric
mu = train.mean(axis=(0)) # general mean
std = train.std(axis=(0)) # sd
train_norm = (train - mu)/std # normalisation
#X_test_norm = (X_test - mu)/std # someday the same to the test
#mu # I checked that all looks good

print(train.info()) # and additional check the previos and norm datasets
print(train_norm.iloc[0:10,0:10])

<class 'pandas.core.frame.DataFrame'>
Index: 2561 entries, HG00096 to NA21144
Columns: 62899 entries, 1:17222:A:G to 22:50807426:C:T
dtypes: int64(62899)
memory usage: 1.2+ GB
None
          1:17222:A:G  1:58771:T:C  1:183401:C:G  1:186291:G:A  1:281912:C:G  \
SampleID                                                                       
HG00096     -0.406959     0.666236      -0.39052     -0.350332     -0.598513   
HG00099     -0.406959    -0.894816      -0.39052      2.808825     -0.598513   
HG00100     -0.406959     0.666236      -0.39052     -0.350332     -0.598513   
HG00101      2.456289     0.666236      -0.39052     -0.350332      1.312698   
HG00102     -0.406959     2.227287      -0.39052     -0.350332      1.312698   
HG00103     -0.406959    -0.894816      -0.39052     -0.350332     -0.598513   
HG00105     -0.406959     0.666236      -0.39052     -0.350332      1.312698   
HG00107     -0.406959     0.666236      -0.39052     -0.350332      1.312698   
HG00108     -0.4069

In [22]:

# workarond: PCA doesn't work since some SNPs had NaN after norm, it seemed they had 0|0 across all samples in train
# but I chose the SNPs used train... anyway, I decided to simple remoce them since it's not many of them

vec = train_norm.isna().any() # the vector contains info if the column contains NaN
vec_T = vec[vec == True] # only TRUE
print(vec_T.index, len(vec_T.index)) # their names and amount

Index(['1:10248540:T:A', '4:22686119:T:A', '4:174674903:T:G', '6:87801487:G:T',
       '6:88803218:T:A', '8:129694056:G:A', '9:27548938:T:C', '9:99352025:C:A',
       '9:121542334:A:T', '11:56953217:A:C', '14:57763802:G:A',
       '15:90603252:C:A', '22:39351776:G:C'],
      dtype='object') 13


In [23]:
#np.any(np.isnan(train_norm)) #
#train.loc[:,'8:15424631:C:G'].mean() # just look inside one of these SNPs

In [24]:

# remove these bad snps from our data

train_norm = train_norm.drop(vec_T.index, axis=1)
train_norm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2561 entries, HG00096 to NA21144
Columns: 62886 entries, 1:17222:A:G to 22:50807426:C:T
dtypes: float64(62886)
memory usage: 1.2+ GB


In [25]:
train_norm

,1:17222:A:G,1:58771:T:C,1:183401:C:G,1:186291:G:A,1:281912:C:G,1:531693:C:T,1:663390:T:C,1:689966:T:C,1:734153:G:C,1:813885:G:A,...,22:50509881:A:G,22:50555369:G:C,22:50592780:G:C,22:50665742:A:G,22:50717650:G:A,22:50740572:C:A,22:50768105:T:C,22:50768106:T:G,22:50787916:C:T,22:50807426:C:T
SampleID,,,,,,,,,,,,,,,,,,,,,
HG00096,-0.406959,0.666236,-0.39052,-0.350332,-0.598513,1.997468,-0.46609,-0.368702,-0.447613,0.132019,...,-1.366794,1.382788,1.410910,1.382368,-1.409589,-0.021309,0.055892,-0.112465,0.207548,-0.370348
HG00099,-0.406959,-0.894816,-0.39052,2.808825,-0.598513,0.499952,-0.46609,-0.368702,-0.447613,1.583089,...,1.345612,-1.416674,-0.003867,0.018637,-0.025403,-1.385607,1.459219,-0.112465,0.207548,-0.370348
HG00100,-0.406959,0.666236,-0.39052,-0.350332,-0.598513,0.499952,-0.46609,-0.368702,-0.447613,1.583089,...,1.345612,-1.416674,-0.003867,1.382368,-0.025403,-1.385607,0.055892,-0.112465,-1.217465,-0.370348
HG00101,2.456289,0.666236,-0.39052,-0.350332,1.312698,0.499952,-0.46609,-0.368702,-0.447613,0.132019,...,-0.010591,-1.416674,-0.003867,0.018637,1.358783,-1.385607,0.055892,-0.112465,0.207548,-0.370348
HG00102,-0.406959,2.227287,-0.39052,-0.350332,1.312698,-0.997565,-0.46609,-0.368702,2.039017,1.583089,...,1.345612,-0.016943,-0.003867,0.018637,-0.025403,-1.385607,1.459219,-0.112465,-1.217465,-0.370348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA21135,-0.406959,2.227287,-0.39052,-0.350332,-0.598513,1.997468,-0.46609,-0.368702,-0.447613,-1.319052,...,-1.366794,-0.016943,-0.003867,1.382368,-1.409589,1.342990,-1.347435,-0.112465,-1.217465,-0.370348
NA21137,2.456289,0.666236,-0.39052,2.808825,1.312698,1.997468,-0.46609,-0.368702,-0.447613,0.132019,...,-1.366794,-1.416674,-0.003867,1.382368,-1.409589,-0.021309,1.459219,-0.112465,0.207548,-0.370348
NA21142,2.456289,2.227287,-0.39052,-0.350332,-0.598513,0.499952,-0.46609,-0.368702,-0.447613,0.132019,...,-1.366794,-0.016943,-0.003867,0.018637,-1.409589,1.342990,-1.347435,-0.112465,0.207548,2.699107


In [26]:

# EVentually, PCA on train! with 0.9 explained variation

from sklearn.decomposition import PCA
pca_09 = PCA(n_components=0.9)
pca_09.fit(train_norm)
train_norm_new_coord = pca_09.transform(train_norm)

In [70]:
test=bg_dt3[~bg_dt3.index.isin(smpl_list.index)]
test_Y = test.loc[:,'Population'] # get Y stay alone
test_Y = test_Y.to_frame()
test_Y.index = range(0,641) # change to merge it later

test = test.drop('Population', axis = 1) # remove Y from data


test = test.replace(['0|0'], '0')
test = test.replace(['0|1',"1|0"], '1')
test = test.replace(['1|1'], '2')

test = test.apply(pd.to_numeric) # to numeric
test_norm = (test - mu)/std # someday the same to the test
#mu # I checked that all looks good

vec = test_norm.isna().any() # the vector contains info if the column contains NaN
vec_T = vec[vec == True] # only TRUE

test_norm = test_norm.drop(vec_T.index, axis=1)

test_norm_new_coord = pca_09.transform(test_norm)

test_X = pd.DataFrame(test_norm_new_coord) # nparray to df; I just get used to work with df

In [29]:
# do a dataset, add normal colnames

toplot = pd.DataFrame(train_norm_new_coord) # nparray to df; I just get used to work with df
train_X = toplot
toplot.columns = ["PC" + str(x) for x in range(1,1898)] # new colnames since right now colnames are RangeIndices
#train_Y.index.name = None
toplot = toplot.join(train_Y)


In [32]:
fig = (
    ggplot(data = toplot,
          mapping = aes(x = 'PC1', y = 'PC2')) +
    geom_point(aes(colour = 'Population',
                   fill = 'Population')) +
    labs(
        title ='',
        x = 'PC1',
        y = 'PC2',
    ) + 
    theme(figure_size = (15, 10)) 
).draw()


In [83]:
#nn,rf/svm, knn/regressioon? logistic
#balance the classes in the training/test
train_yy = np.array(train_Y["Population"])
test_yy = np.array(test_Y["Population"])

In [77]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)

knn.fit(train_X, train_yy)

prediction = knn.predict(test_X)



n_correct = np.sum(test_yy== prediction)
knn_accuracy = n_correct/len(test_Y)
knn_accuracy

C:\Users\annilo\Anaconda3\envs\Tehisintellekt\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names


0.5881435257410297

In [78]:
logisticRegr = LogisticRegression(solver = 'lbfgs')
##### YOUR CODE STARTS #####
logisticRegr.fit(train_X,train_yy)
predictions = logisticRegr.predict(X = test_X)
score = np.sum(predictions == test_yy)/len(test_yy)
##### YOUR CODE ENDS ##### 
print(score)

0.9251170046801872


C:\Users\annilo\Anaconda3\envs\Tehisintellekt\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
C:\Users\annilo\Anaconda3\envs\Tehisintellekt\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names


In [91]:
whole_ds=bg_dt3
whole_ds_Y = whole_ds.loc[:,'Population'] # get Y stay alone
whole_ds_Y = whole_ds_Y.to_frame()
whole_ds_Y.index = range(0,3202) # change to merge it later

whole_ds = whole_ds.drop('Population', axis = 1) # remove Y from data


whole_ds = whole_ds.replace(['0|0'], '0')
whole_ds = whole_ds.replace(['0|1',"1|0"], '1')
whole_ds = whole_ds.replace(['1|1'], '2')

whole_ds = whole_ds.apply(pd.to_numeric) # to numeric
whole_ds_norm = (whole_ds - mu)/std # someday the same to the test
#mu # I checked that all looks good

vec = whole_ds_norm.isna().any() # the vector contains info if the column contains NaN
vec_T = vec[vec == True] # only TRUE

whole_ds_norm = whole_ds_norm.drop(vec_T.index, axis=1)

whole_ds_norm_new_coord = pca_09.transform(whole_ds_norm)

whole_ds_X = pd.DataFrame(whole_ds_norm_new_coord)

whole_ds_yy = np.array(whole_ds_Y["Population"])

In [94]:
model = LinearDiscriminantAnalysis()
# define model evaluation method
cv = RepeatedStratifiedKFold(n_splits=15, n_repeats=5, random_state=1)
scores = cross_val_score(model, train_X, train_yy, scoring='accuracy', cv=cv, n_jobs=-1)
# summarize result


In [95]:
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Mean Accuracy: 0.888 (0.020)
